# 연습 - 종목정보(코드,가격), 계좌잔고확인, 주문, 스케쥴링

## 주의사항
-------------------------------------
* 파이썬은 3.6 32비트 버전을 설치
* 프로젝트 경로 및 폴더명은 공백과 한글이 없도록 설치
* 사이보스는 관리자 권한으로 수행
* 주피터 노트북이 너무 느리면, VS Code에서 파이썬 플러그인을 설치해서 개발
* VS Code Python plugin에서 주피터 서버를 띄워줌. 수행속도는 차이가 없으나 코딩화면에서 반응 속도 빠름.
-------------------------------------

## 종목정보

- [x] @mentions, #refs, [links](), **formatting**, and <del>tags</del> supported
- [x] list syntax required (any unordered or ordered list supported)
- [x] this is a complete item
- [ ] this is an incomplete item



-------------------------------------

	종목번호	F-KOSDAQ150	(연결선물코드 10100)
			
	현재 시각	2020/07/28 14:22:00
	갱신 시각	2020/07/28 14:01:38
			
	20일 종가 내역	Date	Close
		2020/06/30 15:30:00	1256.61
		...
		2020/07/27 15:30:00	1357.23
			
	20일 이동 평균 (종가기준)		1305.58
			
	시초가		1365.03
	최고가		1379.11
	최저가		1362.54
	현재가		1369.69
	IBS     (일일변동폭 대비 종가 수준)		43.15
			
	월모멘텀 : 20이평보다 현재가가 비싼가	T
	단기하락 : IBS가 현재 25%이하인가	    F
	-----------------------------------------
	매수여부		보류


In [3]:
import sys
from PyQt5.QtWidgets import *
import win32com.client
import ctypes
import datetime
 
g_objCodeMgr = win32com.client.Dispatch('CpUtil.CpCodeMgr')
g_objCpStatus = win32com.client.Dispatch('CpUtil.CpCybos')
g_objCpTrade = win32com.client.Dispatch('CpTrade.CpTdUtil')
g_objFutureMgr = win32com.client.Dispatch("CpUtil.CpKFutureCode")
g_objFutureMaster = win32com.client.Dispatch("Dscbo1.FutureMst")

def InitPlusCheck():
    # 프로세스가 관리자 권한으로 실행 여부
    if ctypes.windll.shell32.IsUserAnAdmin():
        print('정상: 관리자권한으로 실행된 프로세스입니다.')
    else:
        print('오류: 일반권한으로 실행됨. 관리자 권한으로 실행해 주세요')
        return False
 
    # 연결 여부 체크
    if (g_objCpStatus.IsConnect == 0):
        print("PLUS가 정상적으로 연결되지 않음. ")
        return False
 
    # 주문 관련 초기화
    ret = g_objCpTrade.TradeInit(0)
    if (ret != 0):
        print("주문 초기화 실패, 오류번호 ", ret)
        return False
 
    return True
 
# CpFutureWeek: 선물 일자별
class CpFutureWeek:
    def __init__(self):
        self.objRq = win32com.client.Dispatch("Dscbo1.FutureWeek1")
 
    def request(self, code, retList):
        self.objRq.SetInputValue(0, code)
 
        datacnt= 0
        while True:
            self.objRq.BlockRequest()
 
            rqStatus = self.objRq.GetDibStatus()
            rqRet = self.objRq.GetDibMsg1()
            if rqStatus != 0:
                print("통신상태", rqStatus, rqRet)
                return False
 
            cnt = self.objRq.GetHeaderValue(0)
 
            for i in range(cnt):
                item = {}
                item['일자'] = self.objRq.GetDataValue(0, i)
                item['시가'] = self.objRq.GetDataValue(1, i)
                item['고가'] = self.objRq.GetDataValue(2, i)
                item['저가'] = self.objRq.GetDataValue(3, i)
                item['종가'] = self.objRq.GetDataValue(4, i)
                item['전일대비'] = self.objRq.GetDataValue(5, i)
                item['누적거래량'] = self.objRq.GetDataValue(6, i)
                item['거래대금'] = self.objRq.GetDataValue(7, i)
                item['미결제약정'] = self.objRq.GetDataValue(8, i)
 
                retList.append(item)
            # end of for
 
            datacnt += cnt
            if self.objRq.Continue == False :
                break
        # end of while
 
 
        # for item in  retList:
        #     data = ''
        #     for key, value in item.items():
        #         if (type(value) == float):
        #             data += '%s:%.2f' % (key, value)
        #         elif (type(value) == str):
        #             data += '%s:%s' % (key, value)
        #         elif (type(value) == int):
        #             data += '%s:%d' % (key, value)
 
        #         data += ' '
        #     print(data)
        # return True
 
 
if __name__ == "__main__":
    if False == InitPlusCheck() :
        exit()
    print("================================")

    # 20일 이동평균 + IBS 구하기
    code = "10600" #코스닥 연결선물코드
    obj = CpFutureWeek()
    retList = []
    obj.request(code, retList) 

    startAmt  = 0.0
    endAmt    = 0.0
    topAmt    = 0.0
    bottomAmt = 0.0    

    ibs       = 0.0

    count = 0
    ma20sum = 0.0
    for item in  retList:
        data = ''

        for key, value in item.items():
            if key == '종가' or key == '일자':
                if (type(value) == float):
                    data += '%s:%.2f' % (key, value)
                    ma20sum = ma20sum + value
                elif (type(value) == str):
                    data += '%s:%s' % (key, value)
                elif (type(value) == int):
                    data += '%s:%d' % (key, value)
        
            data += ' '


        print(data)
        count = count + 1
        if count >= 20 : break   

        if count == 1: #첫레코드는 당일, 당일의 시가 고가 저가 현재가를 가져옴
            nowDate    = item['일자']
            startAmt   = item['시가'] 
            topAmt     = item['고가']
            bottomAmt  = item['저가'] 
            endAmt     = item['종가'] 
            beforeRate = item['전일대비']
            transCnt   = item['누적거래량'] 
            transAmt   = item['거래대금'] 
            noCnmt     = item['미결제약정']

    ma20 = ma20sum / 20
    ibs  = (endAmt - bottomAmt) / (topAmt - bottomAmt) * 100

    # # 종목증거금율
    # value = g_objCodeMgr.GetStockMarginRate( "000660" )
    # print("000660 증거금율 : ", value)
    # print("==========================================")  



    print("==========================================")  
    # 최근 선물코드, 두번째 선물코드를 가져온다.
    fstKFCode = ''
    sndKFCode = ''
    for i in range(g_objFutureMgr.GetCount()):
        code = g_objFutureMgr.GetData(0, i)
        name = g_objFutureMgr.GetData(1, i)
        if (code[0] == '4') :   # spread skip
            continue
        if (code[0] == '10100') : #연결선물 skip
            continue
        if i == 0 : fstKFCode = code
        if i == 1 : sndKFCode = code        

    # 최근 선물코드로 잔존일을 가져온다.
    g_objFutureMaster.SetInputValue(0, fstKFCode)
    # g_objFutureMaster.SetInputValue(0, '106Q9')
    g_objFutureMaster.BlockRequest()
    rqStatus = g_objFutureMaster.GetDibStatus()
    rqRet = g_objFutureMaster.GetDibMsg1()
    # print("통신상태", rqStatus, rqRet)        
    remainDate = g_objFutureMaster.GetHeaderValue(8) - 1 # 잔존일

    print("당기선물코드         : %s" % fstKFCode)
    print("차기선물코드         : %s" % sndKFCode)
    print("당기선물잔존일       : %d" % remainDate)    

    print("==========================================")   
    # 현재일시
    nowTimeStr = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    print("현재 일시            : %s" % nowTimeStr)

    print("==========================================")   
    print("20일 이동평균(종가)               :%.2f" % ma20)
    print("==========================================")   
    print("당일 최고가                       :%.2f" % topAmt)
    print("당일 최저가                       :%.2f" % bottomAmt)
    print("당일 현재가                       :%.2f" % endAmt)
    print("==========================================")
    print("IBS(일일변동폭 대비 현재가 수준)  :%7.2f" % ibs)
    print("==========================================")
    f1 = 0.0
    f2 = 0.0
    buyForce = 0
    if ma20 < endAmt : 
        f1 = 1.0
    else:
        f1 = 0.0

    if ibs < 25 : 
        f2 = 1.0 
    else:
        f2 = 0.0
    buyForce = ((f1 + f2) ** 2.321) * 20
    print("월모멘텀(20일 이평 < 현재가)      :   %.2f" % f1)
    print("단기하락(IBS < 25 퍼센트)         :   %.2f" % f2)
    print("매수강도(월모멘텀+단기하락)^2.321 :     %.f" % buyForce)
    print("==========================================")
    

정상: 관리자권한으로 실행된 프로세스입니다.
일자:20200729    종가:1264.70     
일자:20200728    종가:1269.50     
일자:20200727    종가:1253.00     
일자:20200724    종가:1240.20     
일자:20200723    종가:1253.10     
일자:20200722    종가:1236.00     
일자:20200721    종가:1230.30     
일자:20200720    종가:1213.10     
일자:20200717    종가:1216.10     
일자:20200716    종가:1201.40     
일자:20200715    종가:1213.40     
일자:20200714    종가:1207.40     
일자:20200713    종가:1215.30     
일자:20200710    종가:1208.30     
일자:20200709    종가:1205.80     
일자:20200708    종가:1198.70     
일자:20200707    종가:1192.10     
일자:20200706    종가:1187.40     
일자:20200703    종가:1176.50     
일자:20200702    종가:1159.70     
당기선물코드         : 106Q9
차기선물코드         : 106QC
당기선물잔존일       : 43
현재 일시            : 2020-07-29 20:31:23
20일 이동평균(종가)               :1217.10
당일 최고가                       :1271.50
당일 최저가                       :1257.30
당일 현재가                       :1264.70
IBS(일일변동폭 대비 현재가 수준)  :  52.11
월모멘텀(20일 이평 < 현재가)      :   1.00
단기하락(IBS < 25 퍼센트)         :   0.00
매